In [1]:
import pandas as pd
import stride
import datetime
import pytz
import math

date/time range - should be no more then a few hours to limit scale of the queries

In [2]:
datetime_from = pytz.timezone('israel').localize(datetime.datetime(2022, 4, 3, 10))
datetime_to = datetime_from + datetime.timedelta(hours=4)
print('date/time range')
print('from:', datetime_from)
print('to:', datetime_to)

date/time range
from: 2022-04-03 10:00:00+03:00
to: 2022-04-03 14:00:00+03:00


origin gridbox, this is roughly a 100m x 100m box

it's important to have a consistent logic to the grid box, so it can be used recursively

the logic here is each .001 lon/lat is a grid box, which is roughly 100m x 100m

In [3]:
origin_gridbox_lon_from = 34.808
origin_gridbox_lon_to = 34.809
origin_gridbox_lat_from = 31.961
origin_gridbox_lat_to = 31.962
print('origin grid box')
print('lon:', origin_gridbox_lon_from, '-', origin_gridbox_lon_to)
print('lat:', origin_gridbox_lat_from, '-', origin_gridbox_lat_to)

origin grid box
lon: 34.808 - 34.809
lat: 31.961 - 31.962


target gridboxes

In [4]:
target_gridboxes = {
          # lon_from, lon_to, lat_from, lat_to
    'park': [34.802, 34.805, 31.962, 31.965],
    'museum': [34.76, 34.77, 31.98, 31.99],
    'supermarket': [34.818, 34.820, 31.958, 31.963]
}

walking radius / time

In [5]:
# we focus on a single, large walking radius in this notebook for demonstration purposes
walking_radius = 4
# walking time per radius (this is roughly per 100m)
walking_time_seconds = 150

Get all siri ride stops which passed through the (origin grid box + the walking radius) with recorded at time larger than (date/time range start + walking radius x walking time per grid box) and smaller then date/time range end

In [6]:
datetime_from_plus_radius = datetime_from + datetime.timedelta(seconds=walking_time_seconds * walking_radius)
origin_gridbox_lon_from_radius = origin_gridbox_lon_from - (walking_radius-1)*.001
origin_gridbox_lon_to_radius = origin_gridbox_lon_to + walking_radius*0.001
origin_gridbox_lat_from_radius = origin_gridbox_lat_from - (walking_radius-1)*.001
origin_gridbox_lat_to_radius = origin_gridbox_lat_to + walking_radius*0.001
print('origin grid box radius')
print('lon:', origin_gridbox_lon_from_radius, '-', origin_gridbox_lon_to_radius)
print('lat:', origin_gridbox_lat_from_radius, '-', origin_gridbox_lat_to_radius)

origin grid box radius
lon: 34.805 - 34.812999999999995
lat: 31.958 - 31.966


In [7]:
origin_grid_box_siri_ride_stops = list(stride.iterate('/siri_ride_stops/list', {
    'siri_vehicle_location__recorded_at_time_from': datetime_from_plus_radius,
    'siri_vehicle_location__recorded_at_time_to': datetime_to,
    'siri_vehicle_location__lon__greater_or_equal': origin_gridbox_lon_from_radius,
    'siri_vehicle_location__lon__lower_or_equal': origin_gridbox_lon_to_radius,
    'siri_vehicle_location__lat__greater_or_equal': origin_gridbox_lat_from_radius,
    'siri_vehicle_location__lat__lower_or_equal': origin_gridbox_lat_to_radius,
}))
pd.DataFrame(origin_grid_box_siri_ride_stops)

,id,siri_stop_id,siri_ride_id,order,gtfs_stop_id,nearest_siri_vehicle_location_id,siri_stop__code,siri_ride__siri_route_id,siri_ride__journey_ref,siri_ride__scheduled_start_time,...,nearest_siri_vehicle_location__distance_from_journey_start,nearest_siri_vehicle_location__distance_from_siri_ride_stop_meters,gtfs_ride_stop__gtfs_stop_id,gtfs_ride_stop__gtfs_ride_id,gtfs_ride_stop__arrival_time,gtfs_ride_stop__departure_time,gtfs_ride_stop__stop_sequence,gtfs_ride_stop__pickup_type,gtfs_ride_stop__drop_off_type,gtfs_ride_stop__shape_dist_traveled
0,99990224,2534,3905838,11,1315914,205892153,34783,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,...,4829,29,1315914.0,4110092.0,2022-04-03 07:04:10+00:00,2022-04-03 07:04:10+00:00,11.0,0.0,0.0,4804.0
1,99992418,5029,3905987,16,1315330,205896526,33588,1061,2022-04-03-56637569,2022-04-03 06:55:00+00:00,...,5189,19,1315330.0,4133406.0,2022-04-03 07:10:38+00:00,2022-04-03 07:10:38+00:00,16.0,0.0,0.0,5189.0
2,99994848,2568,3905987,17,1315752,205901751,34311,1061,2022-04-03-56637569,2022-04-03 06:55:00+00:00,...,5486,30,1315752.0,4133406.0,2022-04-03 07:11:03+00:00,2022-04-03 07:11:03+00:00,17.0,0.0,0.0,5486.0
3,99995263,2661,3905838,12,1315329,205902597,33587,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,...,5021,43,1315329.0,4110092.0,2022-04-03 07:04:59+00:00,2022-04-03 07:04:59+00:00,12.0,0.0,0.0,4976.0
4,99999823,2570,3904088,32,1315870,205917193,34637,1113,2022-04-03-0,2022-04-03 06:35:00+00:00,...,12311,27,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,100553430,2534,3927342,20,1315914,207017130,34783,1612,2022-04-03-39551017,2022-04-03 10:40:00+00:00,...,6493,34,1315914.0,4113499.0,2022-04-03 10:59:59+00:00,2022-04-03 10:59:59+00:00,20.0,0.0,0.0,6493.0
524,100555436,5029,3926939,16,1315330,207027277,33588,1061,2022-04-03-56637584,2022-04-03 10:40:00+00:00,...,5244,77,1315330.0,4133417.0,2022-04-03 10:55:38+00:00,2022-04-03 10:55:38+00:00,16.0,0.0,0.0,5189.0
525,100555443,2570,3922303,40,1315870,207027286,34637,1064,2022-04-03-56637986,2022-04-03 09:55:00+00:00,...,17055,11,1315870.0,4133497.0,2022-04-03 10:46:59+00:00,2022-04-03 10:46:59+00:00,40.0,0.0,0.0,17055.0
526,100555961,2661,3927342,21,1315329,207028410,33587,1612,2022-04-03-39551017,2022-04-03 10:40:00+00:00,...,6664,53,1315329.0,4113499.0,2022-04-03 11:00:29+00:00,2022-04-03 11:00:29+00:00,21.0,0.0,0.0,6664.0


focus on a single siri_ride on which we will continue the processing, get all the ride stops for this ride sorted by order

In [8]:
siri_rides = {
    siri_ride_stop['siri_ride_id']: {
        'siri_ride_id': siri_ride_stop['siri_ride_id'],
        'siri_ride_stops': []
    }
    for siri_ride_stop in origin_grid_box_siri_ride_stops
}
focus_siri_ride = siri_rides[list(siri_rides.keys())[0]]
for siri_ride_stop in stride.iterate('/siri_ride_stops/list', {
    'siri_vehicle_location__recorded_at_time_from': datetime_from_plus_radius,
    'siri_vehicle_location__recorded_at_time_to': datetime_to,
    'siri_ride_ids': str(focus_siri_ride['siri_ride_id'])
}):
    siri_ride = siri_rides[siri_ride_stop['siri_ride_id']]
    siri_ride['siri_ride_stops'].append(siri_ride_stop)
for siri_ride in siri_rides.values():
    siri_ride['siri_ride_stops'] = sorted(siri_ride['siri_ride_stops'], key=lambda siri_ride: siri_ride['order'])
pd.DataFrame([{
    k: v for k, v in srs.items() if k.startswith('nearest_siri') or k.startswith('siri_ride') or k in ['order']
} for srs in focus_siri_ride['siri_ride_stops']])

,siri_ride_id,order,nearest_siri_vehicle_location_id,siri_ride__siri_route_id,siri_ride__journey_ref,siri_ride__scheduled_start_time,siri_ride__vehicle_ref,siri_ride__updated_first_last_vehicle_locations,siri_ride__first_vehicle_location_id,siri_ride__last_vehicle_location_id,...,siri_ride__gtfs_ride_id,nearest_siri_vehicle_location__siri_snapshot_id,nearest_siri_vehicle_location__siri_ride_stop_id,nearest_siri_vehicle_location__recorded_at_time,nearest_siri_vehicle_location__lon,nearest_siri_vehicle_location__lat,nearest_siri_vehicle_location__bearing,nearest_siri_vehicle_location__velocity,nearest_siri_vehicle_location__distance_from_journey_start,nearest_siri_vehicle_location__distance_from_siri_ride_stop_meters
0,3905838,11,205892153,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67828,99990224,2022-04-03 07:10:18+00:00,34.812069,31.962322,258,0,4829,29
1,3905838,12,205902597,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67830,99995263,2022-04-03 07:12:23+00:00,34.810135,31.962097,258,0,5021,43
2,3905838,13,205923106,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67834,100005287,2022-04-03 07:15:51+00:00,34.802830,31.961390,182,0,5888,97
3,3905838,14,205928261,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67835,100008056,2022-04-03 07:17:24+00:00,34.800240,31.962357,328,0,6135,48
4,3905838,15,205938384,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67837,100010595,2022-04-03 07:19:05+00:00,34.799747,31.965023,10,0,6448,64
5,3905838,16,205953410,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67840,100018057,2022-04-03 07:21:55+00:00,34.799294,31.967928,2,4,6778,21
6,3905838,17,205963241,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67842,100025480,2022-04-03 07:24:25+00:00,34.798103,31.972017,342,36,7201,42
7,3905838,18,205973299,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67844,100030650,2022-04-03 07:26:25+00:00,34.795258,31.976587,308,32,7825,245
8,3905838,20,205988316,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67847,100037738,2022-04-03 07:29:24+00:00,34.788513,31.977654,266,32,8493,8
9,3905838,21,205998366,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67849,100042562,2022-04-03 07:31:09+00:00,34.781387,31.979242,288,25,9139,230


get only the ride stops on this route which occured at or after the origin grid box (as we can't travel back in time)

In [10]:
first_siri_ride_stop_id = None
min_order = None
min_recorded_at_time = None
for srs in origin_grid_box_siri_ride_stops:
    if srs['siri_ride_id'] == focus_siri_ride['siri_ride_id']:
        if min_order is None or min_order > srs['order']:
            min_order = srs['order']
            min_recorded_at_time = srs['nearest_siri_vehicle_location__recorded_at_time']
            first_siri_ride_stop_id = srs['id']
print('min_order:', min_order)
print('min_recorded_at_time:', min_recorded_at_time)
print('first_siri_ride_stop_id:', first_siri_ride_stop_id)
relevant_ride_stops = []
for srs in focus_siri_ride['siri_ride_stops']:
    if srs['order'] >= min_order:
        relevant_ride_stops.append(srs)
pd.DataFrame([{k: v for k, v in srs.items() if k.startswith('nearest_siri') or k.startswith('siri_ride') or k in ['order']} for srs in relevant_ride_stops])

min_order: 11
min_recorded_at_time: 2022-04-03 07:10:18+00:00
first_siri_ride_stop_id: 99990224


,siri_ride_id,order,nearest_siri_vehicle_location_id,siri_ride__siri_route_id,siri_ride__journey_ref,siri_ride__scheduled_start_time,siri_ride__vehicle_ref,siri_ride__updated_first_last_vehicle_locations,siri_ride__first_vehicle_location_id,siri_ride__last_vehicle_location_id,...,siri_ride__gtfs_ride_id,nearest_siri_vehicle_location__siri_snapshot_id,nearest_siri_vehicle_location__siri_ride_stop_id,nearest_siri_vehicle_location__recorded_at_time,nearest_siri_vehicle_location__lon,nearest_siri_vehicle_location__lat,nearest_siri_vehicle_location__bearing,nearest_siri_vehicle_location__velocity,nearest_siri_vehicle_location__distance_from_journey_start,nearest_siri_vehicle_location__distance_from_siri_ride_stop_meters
0,3905838,11,205892153,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67828,99990224,2022-04-03 07:10:18+00:00,34.812069,31.962322,258,0,4829,29
1,3905838,12,205902597,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67830,99995263,2022-04-03 07:12:23+00:00,34.810135,31.962097,258,0,5021,43
2,3905838,13,205923106,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67834,100005287,2022-04-03 07:15:51+00:00,34.802830,31.961390,182,0,5888,97
3,3905838,14,205928261,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67835,100008056,2022-04-03 07:17:24+00:00,34.800240,31.962357,328,0,6135,48
4,3905838,15,205938384,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67837,100010595,2022-04-03 07:19:05+00:00,34.799747,31.965023,10,0,6448,64
5,3905838,16,205953410,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67840,100018057,2022-04-03 07:21:55+00:00,34.799294,31.967928,2,4,6778,21
6,3905838,17,205963241,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67842,100025480,2022-04-03 07:24:25+00:00,34.798103,31.972017,342,36,7201,42
7,3905838,18,205973299,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67844,100030650,2022-04-03 07:26:25+00:00,34.795258,31.976587,308,32,7825,245
8,3905838,20,205988316,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67847,100037738,2022-04-03 07:29:24+00:00,34.788513,31.977654,266,32,8493,8
9,3905838,21,205998366,1523,2022-04-03-39551844,2022-04-03 06:50:00+00:00,7517469,2022-04-03 09:01:55.842661+00:00,205786732,206213028,...,4110092,67849,100042562,2022-04-03 07:31:09+00:00,34.781387,31.979242,288,25,9139,230


check if within those ride stops we get a hit in one of the target grid boxes

In [11]:
for srs in relevant_ride_stops:
    srs_lon = srs['nearest_siri_vehicle_location__lon']
    srs_lat = srs['nearest_siri_vehicle_location__lat']
    for name, tgb in target_gridboxes.items():
        tgb_lon_from, tgb_lon_to, tgb_lat_from, tgb_lat_to = tgb
        if tgb_lon_from <= srs_lon <= tgb_lon_to and tgb_lat_from <= srs_lat <= tgb_lat_to:
            hit_ride_stops = [
                {'type': 'walk', 'radius': walking_radius},
                {'type': 'siri_ride_stop', 'siri_ride_stop_id': first_siri_ride_stop_id},
                {'type': 'siri_ride_stop', 'siri_ride_stop_id': srs['id']}
            ]
            print('hit', name, hit_ride_stops)

hit museum [{'type': 'walk', 'radius': 4}, {'type': 'siri_ride_stop', 'siri_ride_stop_id': 99990224}, {'type': 'siri_ride_stop', 'siri_ride_stop_id': 100057614}]
hit museum [{'type': 'walk', 'radius': 4}, {'type': 'siri_ride_stop', 'siri_ride_stop_id': 99990224}, {'type': 'siri_ride_stop', 'siri_ride_stop_id': 100060129}]
hit museum [{'type': 'walk', 'radius': 4}, {'type': 'siri_ride_stop', 'siri_ride_stop_id': 99990224}, {'type': 'siri_ride_stop', 'siri_ride_stop_id': 100065076}]
hit museum [{'type': 'walk', 'radius': 4}, {'type': 'siri_ride_stop', 'siri_ride_stop_id': 99990224}, {'type': 'siri_ride_stop', 'siri_ride_stop_id': 100070196}]


ok, we got some direct hits

but now we want more hits from bus changes too, so we will map each ride stop to a grid box which we can then use to call this algo again to get hits from that grid box as origin

In [12]:
processed_gridboxes = set()
for srs in relevant_ride_stops:
    srs_lon = srs['nearest_siri_vehicle_location__lon']
    srs_lat = srs['nearest_siri_vehicle_location__lat']
    if not (origin_gridbox_lon_from <= srs_lon <= origin_gridbox_lon_to and origin_gridbox_lat_from <= srs_lat <= origin_gridbox_lat_to):
        # this ride stop is not within the origin grid box
        lon_from = math.floor(srs_lon*1000)/1000
        lat_from = math.floor(srs_lat*1000)/1000
                 # lon_from, lon_to,        lat_from, lat_to
        gridbox = lon_from, lon_from+0.001, lat_from, lat_from+0.001
        if gridbox not in processed_gridboxes:
            processed_gridboxes.add(gridbox)
            print('process gridbox', gridbox, srs['nearest_siri_vehicle_location__recorded_at_time'])

process gridbox (34.812, 34.812999999999995, 31.962, 31.963) 2022-04-03 07:10:18+00:00
process gridbox (34.81, 34.811, 31.962, 31.963) 2022-04-03 07:12:23+00:00
process gridbox (34.802, 34.803, 31.961, 31.962) 2022-04-03 07:15:51+00:00
process gridbox (34.8, 34.800999999999995, 31.962, 31.963) 2022-04-03 07:17:24+00:00
process gridbox (34.799, 34.8, 31.965, 31.966) 2022-04-03 07:19:05+00:00
process gridbox (34.799, 34.8, 31.967, 31.968) 2022-04-03 07:21:55+00:00
process gridbox (34.798, 34.799, 31.972, 31.973000000000003) 2022-04-03 07:24:25+00:00
process gridbox (34.795, 34.796, 31.976, 31.977) 2022-04-03 07:26:25+00:00
process gridbox (34.788, 34.788999999999994, 31.977, 31.978) 2022-04-03 07:29:24+00:00
process gridbox (34.781, 34.782, 31.979, 31.98) 2022-04-03 07:31:09+00:00
process gridbox (34.778, 34.778999999999996, 31.979, 31.98) 2022-04-03 07:34:25+00:00
process gridbox (34.775, 34.775999999999996, 31.98, 31.981) 2022-04-03 07:34:54+00:00
process gridbox (34.769, 34.7699999999